In [1]:
import os
import numpy as np
import pandas as pd

In [2]:

file_path_1 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/fundamental_all.csv' # Fundamental data
df1 = pd.read_csv(file_path_1)
df1['datadate'] = pd.to_datetime(df1['datadate'])
df1 = df1.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr','costat'])

df1 = df1.sort_values(by=['tic', 'datadate'])

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17745 entries, 1236 to 15067
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     17745 non-null  int64         
 1   datadate  17745 non-null  datetime64[ns]
 2   tic       17745 non-null  object        
 3   actq      13578 non-null  float64       
 4   ancq      13192 non-null  float64       
 5   epsfxq    16905 non-null  float64       
 6   lctq      13579 non-null  float64       
 7   ltq       15674 non-null  float64       
 8   revtq     17110 non-null  float64       
 9   ugiq      496 non-null    float64       
 10  capxy     13254 non-null  float64       
 11  prccq     17377 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 1.8+ MB


In [3]:
df1.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,ugiq,capxy,prccq
1236,1690,1979-12-31,AAPL,23.052,1.584,0.06,11.689,12.262,19.540,NaN,NaN,NaN
1237,1690,1980-03-31,AAPL,28.367,1.986,0.06,14.244,14.632,23.550,NaN,NaN,NaN
1238,1690,1980-06-30,AAPL,37.930,3.777,0.06,20.976,21.926,32.569,NaN,NaN,NaN
1239,1690,1980-09-30,AAPL,54.106,11.244,0.07,37.780,39.401,41.467,NaN,NaN,NaN
1240,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,NaN,34.124969


In [4]:
file_path_2 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/ratios_all.csv' # Financial ratios data
df2 = pd.read_csv(file_path_2)
df2['qdate'] = pd.to_datetime(df2['qdate'])
df2['public_date'] = pd.to_datetime(df2['public_date'])

# Sort by qdate and public_date to ensure the most recent public_date comes last
df2 = df2.sort_values(by=['qdate', 'public_date'])
# Drop duplicates to keep only the first public_date for each qdate
df2 = df2.drop_duplicates(subset=['qdate','TICKER'], keep='first')

columns_to_keep = [
    'gvkey', 'qdate', 'roa',
    'roe','pe_inc', 'divyield',
    'pcf', 'roce', 'fcf_ocf',
    'debt_ebitda', 'gpm', 'opmad',
    'evm', 'intcov_ratio', 'at_turn',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb',
]

df2 = df2[columns_to_keep]

# convert divyield to float
df2['divyield'] = df2['divyield'].replace('%', '', regex=True).astype(float)

df2 = df2.sort_values(by=['qdate'])

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15533 entries, 0 to 26692
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gvkey         15533 non-null  int64         
 1   qdate         15520 non-null  datetime64[ns]
 2   roa           15472 non-null  float64       
 3   roe           15267 non-null  float64       
 4   pe_inc        14787 non-null  float64       
 5   divyield      11974 non-null  float64       
 6   pcf           15467 non-null  float64       
 7   roce          15353 non-null  float64       
 8   fcf_ocf       14206 non-null  float64       
 9   debt_ebitda   15355 non-null  float64       
 10  gpm           15512 non-null  float64       
 11  opmad         15512 non-null  float64       
 12  evm           15437 non-null  float64       
 13  intcov_ratio  14432 non-null  float64       
 14  at_turn       15475 non-null  float64       
 15  cfm           15197 non-null  float6

In [5]:
df = pd.merge(df1, df2, left_on=['datadate', 'gvkey'], right_on=['qdate', 'gvkey'], how='inner')

columns_to_keep = [
    'gvkey', 'datadate', 'tic',
    'actq', 'ancq', 'epsfxq',
    'lctq', 'ltq', 'revtq',
    'capxy', 'prccq', 'roa',
    'roe','pe_inc', 'divyield',
    'pcf', 'roce', 'debt_ebitda',
    'gpm', 'evm', 'cfm', 'npm',
    'PEG_trailing', 'de_ratio',
    'curr_ratio', 'ptb',
]

column_renames = {
    'datadate': 'date', 
    'PRC': 'price', 
    'epsfxq': 'earnings_per_share', 
    'ltq': 'long_term_debt', 
    'niq': 'net_income', 
    'revtq': 'revenue', 
    'capxy': 'capital_expenditure', 
    'npm': 'net_profit_margin', 
    'de_ratio': 'debt/equity_Ratio', 
    'curr_ratio': 'current_ratio', 
    'ptb': 'price/book_ratio',
    'tic': 'ticker',
    'actq': 'current_assets',
    'ancq': 'non_current_assets',
    'lctq': 'total_liability',
    'prccq': 'price',
    'roa': 'return_on_assets',
    'roe': 'return_on_equity',
    'pe_inc': 'price/equity_ratio',
    'divyield': 'dividend_yield',
    'pcf': 'price/cash_flow',
    'roce': 'return_on_capital_employed',
    'debt_ebitda': 'debt/ebitda_ratio',
    'gpm': 'gross_profit_margin',
    'evm': 'enterprise_value_multiple',
    'cfm': 'cash_flow_margin',
    'PEG_trailing': 'trailing_p/e_to_growth_ratio'
}

df = df[columns_to_keep]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15523 entries, 0 to 15522
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gvkey         15523 non-null  int64         
 1   datadate      15523 non-null  datetime64[ns]
 2   tic           15523 non-null  object        
 3   actq          12739 non-null  float64       
 4   ancq          12361 non-null  float64       
 5   epsfxq        15404 non-null  float64       
 6   lctq          12742 non-null  float64       
 7   ltq           14742 non-null  float64       
 8   revtq         15076 non-null  float64       
 9   capxy         12419 non-null  float64       
 10  prccq         15502 non-null  float64       
 11  roa           15469 non-null  float64       
 12  roe           15264 non-null  float64       
 13  pe_inc        14789 non-null  float64       
 14  divyield      11974 non-null  float64       
 15  pcf           15464 non-null  float6

In [6]:
df = df.sort_values(by=['tic','datadate'])

df.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,...,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb
0,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,...,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083
1,1690,1981-03-31,AAPL,173.152,17.587,0.16,39.428,43.693,78.765,NaN,...,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188
2,1690,1981-06-30,AAPL,201.262,21.085,0.21,49.914,58.268,90.719,NaN,...,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507
3,1690,1981-09-30,AAPL,227.104,27.734,0.19,70.280,77.451,97.678,NaN,...,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659
4,1690,1981-12-31,AAPL,249.160,36.026,0.24,83.952,92.894,133.553,NaN,...,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086


In [7]:
file_path_3 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/s&p500index.csv' # s&p500
df3 = pd.read_csv(file_path_3)
df3['caldt'] = pd.to_datetime(df3['caldt'])  # Adjust column name if needed

df = df.sort_values(by=['datadate'])


#merge (fill empty cells with nearest s&p500 value)
df = pd.merge_asof(df, df3, left_on='datadate', right_on='caldt', direction='nearest')
df = df.drop(columns=['caldt'])

# calculate relative returns
#df['relative_quarterly_return'] = (df['prccq'] - df['spindx'])
#df = df.drop(columns=['prccq', 'spindx'])

#df = df.dropna(subset=['prccq'])

df = df.sort_values(by=['tic','datadate']).reset_index()
df.head()

,index,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,...,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,spindx
0,2010,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,...,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,135.76
1,2067,1690,1981-03-31,AAPL,173.152,17.587,0.16,39.428,43.693,78.765,...,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,136.00
2,2115,1690,1981-06-30,AAPL,201.262,21.085,0.21,49.914,58.268,90.719,...,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,131.21
3,2132,1690,1981-09-30,AAPL,227.104,27.734,0.19,70.280,77.451,97.678,...,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,116.18
4,2208,1690,1981-12-31,AAPL,249.160,36.026,0.24,83.952,92.894,133.553,...,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,122.55


In [8]:


# convert features to pct

features = [
    'actq', 'ancq', 'epsfxq',
    'lctq', 'ltq', 'revtq',
    'capxy']

ratios = [
    'roa', 'roe','pe_inc', 'divyield',
    'pcf', 'roce',
    'debt_ebitda', 'gpm','evm',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb']

df[features] = df[features].pct_change()

for col in ratios:
    df[col + '_change'] = df[col].pct_change()


df['prccq_change'] =  df['prccq'].pct_change()
df['next_prccq_change'] = df['prccq_change'].shift(-1)

df['spindx_change'] = df['spindx'].pct_change()
df['next_spindx_change'] = df['spindx_change'].shift(-1)

df['relative_quarterly_return'] = (df['prccq_change'] - df['spindx_change'])
df['next_relative_quarterly_return'] = df['relative_quarterly_return'].shift(-1)

#df = df.drop(columns=['prccq', 'spindx'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15523 entries, 0 to 15522
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   index                           15523 non-null  int64         
 1   gvkey                           15523 non-null  int64         
 2   datadate                        15523 non-null  datetime64[ns]
 3   tic                             15523 non-null  object        
 4   actq                            15522 non-null  float64       
 5   ancq                            15522 non-null  float64       
 6   epsfxq                          15521 non-null  float64       
 7   lctq                            15522 non-null  float64       
 8   ltq                             15522 non-null  float64       
 9   revtq                           15522 non-null  float64       
 10  capxy                           14963 non-null  float64       
 11  pr

In [9]:
# REMOVE FALSE VALUES CAUSED BY PCT CHANGE

# Create a list to track the indices to drop
indices_to_drop = []

# Initialize the previous ticker as None
previous_ticker = None

# Iterate through the DataFrame, skipping the first row
for idx, row in df.iterrows():
    current_ticker = row['tic']
    
    # If the current ticker is different from the previous one, mark the current row for deletion
    if previous_ticker is not None and current_ticker != previous_ticker:
        indices_to_drop.append(idx)  # Append the index of the first occurrence of the new ticker
        indices_to_drop.append(idx-1)  # Append the index of the first occurrence of the new ticker
    
    # Update the previous ticker
    previous_ticker = current_ticker

# Drop the rows with the indices in the list
df = df.drop(indices_to_drop)

# Reset the index of the DataFrame after dropping rows
df = df.reset_index(drop=True)

df = df.drop(columns = ['index'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   gvkey                           15339 non-null  int64         
 1   datadate                        15339 non-null  datetime64[ns]
 2   tic                             15339 non-null  object        
 3   actq                            15338 non-null  float64       
 4   ancq                            15338 non-null  float64       
 5   epsfxq                          15337 non-null  float64       
 6   lctq                            15338 non-null  float64       
 7   ltq                             15338 non-null  float64       
 8   revtq                           15338 non-null  float64       
 9   capxy                           14787 non-null  float64       
 10  prccq                           15338 non-null  float64       
 11  ro

In [10]:

df = df.sort_values(by=['tic','datadate'])

df_temp = df[['spindx', 'spindx_change', 'next_spindx_change', 'prccq', 'prccq_change', 'next_prccq_change']]

df.head()


#df.to_csv('processed_data.csv', index=False)  # Set index=False to exclude the index column

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,...,PEG_trailing_change,de_ratio_change,curr_ratio_change,ptb_change,prccq_change,next_prccq_change,spindx_change,next_spindx_change,relative_quarterly_return,next_relative_quarterly_return
0,1690,1980-12-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.282051,NaN,0.001768,NaN,-0.283819
1,1690,1981-03-31,AAPL,0.188251,0.072247,0.142857,0.365898,0.404152,0.164801,NaN,...,NaN,-0.244604,0.238132,0.099702,-0.282051,0.061225,0.001768,-0.035221,-0.283819,0.096445
2,1690,1981-06-30,AAPL,0.162343,0.198897,0.312500,0.265953,0.333577,0.151768,NaN,...,NaN,-0.123810,0.138219,-0.466114,0.061225,-0.413461,-0.035221,-0.114549,0.096445,-0.298912
3,1690,1981-09-30,AAPL,0.128400,0.315343,-0.095238,0.408022,0.329220,0.076709,NaN,...,NaN,0.187500,-0.122249,-0.130321,-0.413461,0.450819,-0.114549,0.054829,-0.298912,0.395990
4,1690,1981-12-31,AAPL,0.097119,0.298983,0.263158,0.194536,0.199391,0.367278,NaN,...,NaN,-0.084668,0.080780,-0.101255,0.450819,-0.237288,0.054829,-0.086414,0.395990,-0.150875


In [11]:
file_path_4 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/pro_data.csv' # cleaned data
df = pd.read_csv(file_path_4)
df['datadate'] = pd.to_datetime(df['datadate'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   gvkey                           15339 non-null  int64         
 1   datadate                        15339 non-null  datetime64[ns]
 2   tic                             15339 non-null  object        
 3   actq                            15338 non-null  float64       
 4   ancq                            15338 non-null  float64       
 5   epsfxq                          15337 non-null  float64       
 6   lctq                            15338 non-null  float64       
 7   ltq                             15338 non-null  float64       
 8   revtq                           15338 non-null  float64       
 9   capxy                           14787 non-null  float64       
 10  roa                             15297 non-null  float64       
 11  ro

In [12]:
df[['spindx', 'spindx_change', 'next_spindx_change', 'prccq', 'prccq_change', 'next_prccq_change']] = df_temp

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   gvkey                           15339 non-null  int64         
 1   datadate                        15339 non-null  datetime64[ns]
 2   tic                             15339 non-null  object        
 3   actq                            15338 non-null  float64       
 4   ancq                            15338 non-null  float64       
 5   epsfxq                          15337 non-null  float64       
 6   lctq                            15338 non-null  float64       
 7   ltq                             15338 non-null  float64       
 8   revtq                           15338 non-null  float64       
 9   capxy                           14787 non-null  float64       
 10  roa                             15297 non-null  float64       
 11  ro

In [13]:
# Replace inf and -inf with NaN
df.replace(['inf', '-inf', np.inf, -np.inf], np.nan, inplace=True)

df.tail()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,...,curr_ratio_change,ptb_change,relative_quarterly_return,next_relative_quarterly_return,spindx,spindx_change,next_spindx_change,prccq,prccq_change,next_prccq_change
15334,13721,2022-09-30,ZTS,-0.015035,0.000562,0.008929,-0.056047,-0.019369,-0.024355,0.590038,...,-0.134967,-0.032525,-0.084526,-0.082539,3585.62,-0.052771,0.070805,148.29,-0.137297,-0.011734
15335,13721,2022-12-31,ZTS,0.145779,0.041556,-0.123894,0.099653,0.167554,0.018972,0.412048,...,-0.054270,0.165286,-0.082539,0.065449,3839.50,0.070805,0.070272,146.55,-0.011734,0.135722
15336,13721,2023-03-31,ZTS,-0.164801,0.008896,0.202020,-0.395327,-0.119654,-0.003430,-0.619454,...,0.033755,-0.043011,0.065449,-0.048332,4109.31,0.070272,0.082999,166.44,0.135722,0.034667
15337,13721,2023-06-30,ZTS,-0.013718,0.010822,0.218487,-0.076240,-0.014574,0.072271,0.744395,...,0.111837,0.134524,-0.048332,0.046754,4450.38,0.082999,-0.036476,172.21,0.034667,0.010278
15338,13721,2023-09-30,ZTS,0.006308,0.042030,-0.110345,-0.091012,-0.010517,-0.013297,0.372751,...,0.136197,-0.161976,0.046754,NaN,4288.05,-0.036476,NaN,173.98,0.010278,NaN


In [14]:
#df.to_csv('dati_trattati.csv', index=False)  # Set index=False to exclude the index column